# Package model hosted in curated registries

In this examples, you will learn how to package models hosted in curated registries.

## Prerequisites

Ensure you have the latest version of `azure-ai-ml`:

```bash
%pip install -U azure-ai-ml
```

## 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

### 1.1. Import the required libraries

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    AzureMLOnlineInferencingServer,
    ModelPackage,
    ModelConfiguration,
)
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Environment,
    Model,
)
from azure.ai.ml.constants import AssetTypes

### 1.2 Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
subscription_id = "<subscription>"
resource_group = "<resource-group>"
workspace = "<workspace>"

In [ ]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

If you are running on AzureML compute, you can easily:

In [ ]:
ml_client = MLClient.from_config(DefaultAzureCredential())

## 2. Getting the model

Let's get a model reference from a model in the catalog:

In [ ]:
model_name = "heart-classifier-mlflow"
model_path = "model"

model = ml_client.models.create_or_update(
    Model(name=model_name, path=model_path, type=AssetTypes.MLFLOW_MODEL)
)

In [ ]:
model = ml_client.models.get(name=model_name, label="latest")

## 3. Package the model

Let's package this model for online deployment

### 3.1 The base environment

Packages uses a base environment to construct the package. However, for MLflow models, we automatically select the best base image depending on the SKU of your target compute.

### 3.2 Package the model

In [ ]:
pakage_config = ModelPackage(
    target_environment="heart-classifier-mlflow-pkg",
    inferencing_server=AzureMLOnlineInferencingServer(),
)

Let's start the package operation:

In [ ]:
model_package = ml_client.models.package(model_name, model.version, pakage_config)

The package operation will start and it will take a couple of minutes to complete. You can get the details of this package with:

In [ ]:
model_package

> Notice how the package operation results in a new environment version being created.

## 4. Deploy the package to Online Endpoints

Now, we can deploy this package in an Online Endpoint.

### 4.1 Create the endpoint

Let's name the endpoint:

In [ ]:
endpoint_name = "heart-classifier-mlflow"

Endpoint names should be unique so we will append a random string at the end to ensure that:

In [ ]:
import random
import string

# Creating a unique endpoint name by including a random suffix
allowed_chars = string.ascii_lowercase + string.digits
endpoint_suffix = "".join(random.choice(allowed_chars) for x in range(5))
endpoint_name = f"{endpoint_name}-{endpoint_suffix}"

print(f"Endpoint name: {endpoint_name}")

Let's create the endpoint:

In [ ]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name)
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

### 4.2 Deploy the package in a deployment

Now, we can deploy this package in an Online Endpoint.

In [ ]:
deployment_name = "with-package"
deployment_package = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=endpoint_name,
    environment=model_package,
    instance_count=1,
)

> Tip: Notice how model or scoring script are not being indicated in this example. All of them are part of the package.

Create the deployment:

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment_package).result()

### 4.3 Test the deployment

We can test if the deployment is working as expected. Once the deployment is created, it is ready to receive requests.

In [ ]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name=deployment_name,
    request_file="sample-request.json",
)

Now that we confirmed the deployment works, let's send all the traffic to it:

In [ ]:
endpoint.traffic = {deployment_name: 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

## 5. Deploying with packages directly from the deployment

If you don't need to configure how the package is performed, you can quickly take advantage of the packaging functionality by indicating Online Endpoints to package before performing the deployment.

To do so, indicate the argument `with_package=True`.

In [ ]:
deployment_package = ManagedOnlineDeployment(
    name="with-package-inline",
    endpoint_name=endpoint_name,
    model=model.id,
    instance_count=1,
    with_package=True,
)

Let's create the deployment:

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment_package).result()

## 6. Deploy the model outside AzureML

Packages also allow you to generate a serving image that can be used to get the model deployed to whatever target is needed. Since AzureML is able to do some optimizations when packaging models, like for instance to mount the location where the model is stored rather than to copy the entire file inside of the image, you need to indicate to AzureML that you intend to take the image outside of the workspace.

Let's see how you can configure the package to do so:

### 6.1 Package the model

We are going to package the model again, but this time, we are going to copy the resources inside of the image so we can take it outside of AzureML.

In [ ]:
pakage_config = ModelPackage(
    target_environment="heart-classifier-mlflow-pkg",
    inferencing_server=AzureMLOnlineInferencingServer(),
    model_configuration=ModelConfiguration(mode="copy"),
)

Let's start the package operation:

In [ ]:
model_package = ml_client.models.package(model_name, model.version, pakage_config)

## 7. Clean un resources

Once done, delete the associated resources from the workspace:

In [ ]:
ml_client.online_endpoints.begin_delete(endpoint.name).result()